In [1]:
import pandas as pd
import plotly
import plotly.plotly as py
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go

import sklearn
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler

import numpy as np

init_notebook_mode(connected=True)

### Задание 1

Implement the kNN algorithm and calculate Leave-one-out error on both datasets for 1 to 10 neighbors.


In [4]:
cancer_df = pd.read_csv('cancer.csv', sep=',', header=0)
spam_df   = pd.read_csv('spam.csv'  , sep=',', header=0)

cancer_df['label'] = cancer_df['label'].apply(lambda x: 1 if x == 'M' else 0)

In [4]:
def find_k_nearest(df, point, k):
    return df.sub(np.array(point)).pow(2).sum(1).pow(0.5).sort_values()[:k]

class kNN:
    def __init__(self, k):
        self.k = k
    
    def learn(self, df):
        self.data_set = df
    
    def predict(self, point):
        nearest = find_k_nearest(self.data_set.drop(columns=['label']), point, self.k)
        zeros_cnt = 0
        ones_cnt  = 0
        last_is_one = None
        for i in range(len(nearest.index)):
            value = self.data_set.at[nearest.index[i], 'label']
            assert value == 0 or value == 1
            if value:
                ones_cnt += 1; last_is_one = True
            else:
                zeros_cnt += 1; last_is_one = False
        assert last_is_one != None
        if zeros_cnt > ones_cnt:
            return 0
        if zeros_cnt < ones_cnt:
            return 1
        assert zeros_cnt == ones_cnt
        # As distances were sorted, let's pick the set that would win for k - 1
        return 0 if last_is_one else 1

def loo(df, alg):
    sum = 0
    for index in df.index:
        alg.learn(df.drop([index]))
        sum += 1 if alg.predict(df.loc[index][1:]) != df.at[index, 'label'] else 0
    return sum / len(df.index)

rang = list(range(1, 11))
cancer_knn_loos = [loo(cancer_df, kNN(k)) for k in rang]
smap_knn_loos = [loo(spam_df, kNN(k)) for k in rang]

In [71]:
cancer_knn_plot = go.Scatter(
    y = cancer_knn_loos,
    x = rang,
    mode = 'lines+markers',
    name = 'lines+markers'
)

plotly.offline.iplot([cancer_knn_plot], filename='line-mode')
plotly.offline.plot([cancer_knn_plot], filename='cancer_knn_plot.html', auto_open=False)

'file:///home/annikura/HSE/ML/1/cancer_knn_plot.html'

In [105]:
spam_knn_plot = go.Scatter(
    y = smap_knn_loos,
    x = rang,
    mode = 'lines+markers',
    name = 'lines+markers'
)

plotly.offline.iplot([spam_knn_plot], filename='line-mode')
plotly.offline.plot([spam_knn_plot], filename='spam_knn_plot.html', auto_open=False)

'file:///home/annikura/HSE/ML/1/spam_knn_plot.html'

### Задание 2

Scale all features with MinMax scaler to [0,1] and calculate Leave-one-out error on both datasets for 1 to 10 neighbors.

In [7]:
scaler = MinMaxScaler()

normalized_cancer_df = pd.DataFrame(scaler.fit_transform(cancer_df), columns=cancer_df.columns)
normalized_spam_df   = pd.DataFrame(scaler.fit_transform(spam_df), columns=spam_df.columns)

norm_cancer_knn_loos = [loo(normalized_cancer_df, kNN(k)) for k in rang]
norm_smap_knn_loos = [loo(normalized_spam_df, kNN(k)) for k in rang]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning:

Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning:

Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.



In [73]:
norm_cancer_knn_plot = go.Scatter(
    y = norm_cancer_knn_loos,
    x = rang,
    mode = 'lines+markers',
    name = 'lines+markers'
)

plotly.offline.iplot([norm_cancer_knn_plot], filename='line-mode')
plotly.offline.plot([norm_cancer_knn_plot], filename='norm_cancer_knn.html', auto_open=False)

'file:///home/annikura/HSE/ML/1/norm_cancer_knn.html'

In [74]:
norm_spam_knn_plot = go.Scatter(
    y = norm_smap_knn_loos,
    x = rang,
    mode = 'lines+markers',
    name = 'lines+markers'
)

plotly.offline.iplot([norm_spam_knn_plot], filename='line-mode')
plotly.offline.plot([norm_spam_knn_plot], filename='norm_spam_knn.html', auto_open=False)

'file:///home/annikura/HSE/ML/1/norm_spam_knn.html'

### Задание 3

 Implement the k-means algorithm and cluster data point into [2,3,4,5] clusters.


In [10]:
blobs_df = pd.read_csv('blobs.csv', sep=',', header=0)

In [11]:
def closest_to(centers, point):
    cluster = -1
    mi = 0
    k = len(centers)
    for j in range(k):
        distance = np.linalg.norm(point - centers[j])
        if cluster == -1 or mi > distance:
            mi = distance
            cluster = j
    assert cluster != -1
    return cluster
        

def kMeans(df, k, iters=1000):
    dim = len(df.columns)
    maxvals = np.array([df[col].max() for col in df.columns])
    minvals = np.array([df[col].min() for col in df.columns])
    centers = [np.random.uniform(minvals, maxvals, dim) for _ in range(k)]
    clusters = None
    clusters_cnt = None
    for _ in range(iters):
        clusters = [np.zeros(dim) for _ in range(k)]
        clusters_cnt = [0] * k
        for i in range(len(df.index)):
            cluster = closest_to(centers, np.array(df.iloc[i]))
            clusters[cluster] += np.array(df.iloc[i])
            clusters_cnt[cluster] += 1
        for i in range(k):
            if clusters_cnt[i] == 0:
                centers[i] = np.random.uniform(minvals, maxvals, dim)
                continue
            centers[i] = clusters[i] / clusters_cnt[i]
    return centers

centers = []
for clusters in range(2, 6):
    centers.append(kMeans(blobs_df, clusters, iters=1000))

In [75]:
for i, cntrs in enumerate(centers):
    i += 2
    kMeans_clusters = []
    for j in range(i):
        mask = pd.Series([closest_to(cntrs, np.array(blobs_df.iloc[k])) == j for k in range(len(blobs_df.index))])
        kMeans_clusters.append(go.Scatter(
            y = blobs_df[mask]['Y'],
            x = blobs_df[mask]['X'],
            mode = 'markers',
            name = 'cluster' + str(j)
        ))
        
    plotly.offline.iplot(kMeans_clusters, filename='line-mode')
    plotly.offline.plot(kMeans_clusters, filename='kmeans_{}.html'.format(i), auto_open=False)

### Задание 4

Implement the DBSCAN algorithm and find parameters for clustering into [2,3,4,5] clusters.


In [77]:
def dbscan(df, eps, minpts):
    vs = []
    clust = [-1] * len(df.index)
    for i in range(len(df.index)):
        if find_k_nearest(df, df.iloc[i], minpts + 1).iloc[minpts] < eps:
            vs.append(i)
    def dfs(u, col):
        clust[u] = col
        for v in vs:
            if clust[v] != -1:
                continue
            if np.linalg.norm(np.array(df.iloc[u]) - np.array(df.iloc[v])) < eps:
                dfs(v, col)
            
    emp = 0      
    for v in vs:
        if clust[v] != -1:
            continue
        dfs(v, emp)
        emp += 1
    return clust

dbscan_clusters = [
    dbscan(blobs_df, 0.5, 40),
    dbscan(blobs_df, 0.5, 43),
    dbscan(blobs_df, 0.25, 10),
    dbscan(blobs_df, 0.2, 10),
]

In [78]:
for cluster in dbscan_clusters:
    dbscan_clusters_plot = []
    for j in range(-1, 6):
        mask = pd.Series([cluster[k] == j for k in range(len(blobs_df.index))])
        dbscan_clusters_plot.append(go.Scatter(
            y = blobs_df[mask]['Y'],
            x = blobs_df[mask]['X'],
            mode = 'markers',
            name = 'cluster' + str(j)
        ))

    plotly.offline.iplot(dbscan_clusters_plot, filename='line-mode')
    plotly.offline.plot(dbscan_clusters_plot, filename='dbscan.html', auto_open=False)
    

last_clusters = None

### Задание 5

 Implement the Agglomerative Clustering and output the clustering into [2,3,4,5] clusters.



In [68]:
def agglomerative_clusterisation_step(df, clusters):
    def cluster_dists(df, a, b):
        dist = -1
        for i in a:
            for j in b:
                    dist = max(dist, np.linalg.norm(np.array(df.iloc[i]) - np.array(df.iloc[j])))
        assert dist != -1
        return dist
        
    sz = len(clusters)
    a = -1
    b = -1
    dist = -1
    for i in range(sz):
        for j in range(i + 1, sz):
            new_dist = cluster_dists(df, clusters[i], clusters[j]) 
            if dist == -1 or new_dist < dist:
                a = i; b = j
                dist = new_dist
    assert dist != -1 and a != -1 and b != -1
    new_clusters = []
    last_clusters = clusters
    for i, clust in enumerate(clusters):
        if i != a and i != b:
            new_clusters.append(clust)
        if i == a:
            new_clusters.append(clust + clusters[b])
    return new_clusters
    
def agglomerative_clusterisation(df, k):
    sz = len(df.index)
    clusters = [[i] for i in range(sz)]
    for ind in range(sz - k):
        clusters = agglomerative_clusterisation_step(df, clusters)
    return clusters

agglomerative_clusters5 = agglomerative_clusterisation(blobs_df, 5)

In [69]:
agglomerative_clusters4 = agglomerative_clusterisation_step(blobs_df, agglomerative_clusters5)
agglomerative_clusters3 = agglomerative_clusterisation_step(blobs_df, agglomerative_clusters4)
agglomerative_clusters2 = agglomerative_clusterisation_step(blobs_df, agglomerative_clusters3)

agglomerative_clusters = [
    agglomerative_clusters2,
    agglomerative_clusters3,
    agglomerative_clusters4,
    agglomerative_clusters5,
]

In [70]:
for cluster in agglomerative_clusters:
    agglomerative_clusters_plot = []
    for j in range(len(cluster)):
        mask = pd.Series([k in cluster[j] for k in range(len(blobs_df.index))])
        agglomerative_clusters_plot.append(go.Scatter(
            y = blobs_df[mask]['Y'],
            x = blobs_df[mask]['X'],
            mode = 'markers',
            name = 'cluster' + str(j)
        ))

    plotly.offline.iplot(agglomerative_clusters_plot, filename='agglomerative_clusters{}.html'.format(len(cluster)))
    plotly.offline.plot(agglomerative_clusters_plot, filename='agglomerative_clusters{}.html'.format(len(cluster)), auto_open=False)

### Задание 6

Cluster datapoints into [2,3,5,10] clusters with k-Means and calculate the Purity metric.


In [16]:
cancer_centers = {}

for k in [2, 3, 5, 10]:
    cancer_centers[k] = kMeans(cancer_df.drop(columns=['label']), k)

In [26]:
def purge_kMeans(centers, df):
        data = df.drop(columns=['label'])
        labs = df['label']
        
        cluster_ones = {}
        cluster_zeros = {}
        
        for i in range(len(df.index)):
            clust = closest_to(centers, np.array(data.iloc[i]))
            if clust not in cluster_ones:
                cluster_ones[clust] = 0
                cluster_zeros[clust] = 0
                
            if labs.iloc[i] == 0:
                cluster_zeros[clust] += 1
            else:
                assert labs.iloc[i] == 1
                cluster_ones[clust] += 1
        
        return sum([max(cluster_zeros[i], cluster_ones[i]) for i in range(len(centers))]) / len(df.index)

In [27]:
for k, cents in cancer_centers.items():
    print("Cancer clasterisation purity for {} clusters: {}".format(k, purge_kMeans(cents, cancer_df)))

Cancer clasterisation purity for 2 clusters: 0.8541300527240774
Cancer clasterisation purity for 3 clusters: 0.8664323374340949
Cancer clasterisation purity for 5 clusters: 0.8681898066783831
Cancer clasterisation purity for 10 clusters: 0.9068541300527241


### Задание 8

Split the dataset into training and validation datasets (80%/20%). Print out the proportions of classes in all datasets.

In [6]:
def split_dataset(df, train_frac):
    # Shuffle
    df = sklearn.utils.shuffle(df)
    sz = len(df.index)
    return df[:int(sz * train_frac)], df[int(sz * train_frac):]

spam_train_set, spam_test_set = split_dataset(spam_df, 0.8)
cancer_train_set, cancer_test_set = split_dataset(cancer_df, 0.8)

In [46]:
print("Spam M proportion: in training set: {} ({} out of {}), in validation set: {} ({} out of {})"
      .format(spam_train_set.sum().at['label'] / len(spam_train_set.index), spam_train_set.sum().at['label'], len(spam_train_set.index), 
              spam_test_set.sum().at['label'] / len(spam_test_set.index), spam_test_set.sum().at['label'], len(spam_test_set.index)))


print("Cancer 1 proportion: in training set: {} ({} out of {}), in validation set: {} ({} out of {})"
      .format(cancer_train_set.sum().at['label'] / len(cancer_train_set.index), cancer_train_set.sum().at['label'], len(cancer_train_set.index), 
              cancer_test_set.sum().at['label'] / len(cancer_test_set.index), cancer_test_set.sum().at['label'], len(cancer_test_set.index)))


Spam M proportion: in training set: 0.3970108695652174 (1461.0 out of 3680), in validation set: 0.38219326818675353 (352.0 out of 921)
Cancer 1 proportion: in training set: 0.367032967032967 (167.0 out of 455), in validation set: 0.39473684210526316 (45.0 out of 114)


### Задание 7

Calculate ROC-AUC for threshold rules for every feature for both datasets. Draw ROC curves for the best three in both.

In [84]:
def roc_features(df):
    sz = len(df.index)
    rocs = {}
    for column in df.drop(columns=['label']):
        fpr, tpr, _ = sklearn.metrics.roc_curve(df['label'], df[column])
        rocs[column] = list(zip(tpr, fpr))
    return rocs

In [100]:
cancer_roc = roc_features(cancer_df)

top_cancer_aucs = sorted([(sklearn.metrics.roc_auc_score(cancer_df['label'], cancer_df[key]), key) for key, _ in cancer_roc.items()], reverse=True)

In [5]:
def draw_roc(roc, name='roc'):
    plot = go.Scatter(
        y = list(map(lambda x: x[0], roc)),
        x = list(map(lambda x: x[1], roc)),
        mode = 'lines',
        name = 'roc'
    )

    plotly.offline.iplot([plot], filename=name + '.html')

In [101]:
for auc, col in top_cancer_aucs[:3]:
    print("Cancer: Column: {}, auc: {}".format(col, auc))
    draw_roc(cancer_roc[col], name='cancer_roc' + col)

Cancer: Column: 23, auc: 0.9754505575815232


Cancer: Column: 21, auc: 0.9704428941387877


Cancer: Column: 24, auc: 0.9698284974367105


In [102]:
spam_roc = roc_features(spam_df)

top_spam_aucs = sorted([(sklearn.metrics.roc_auc_score(spam_df['label'], spam_df[key]), key) for key, _ in spam_roc.items()], reverse=True)

In [62]:
class BinaryDecisionTree:
    class Node:
        def __init__(self, p, left=None, right=None, column=None, trashhold=None):
            self.column = column
            self.trashhold = trashhold
            self.p = p
            self.left = left
            self.right = right
            
        def predict(self, x, maxlvl):
            if self.column is None or maxlvl == 0:
                return self.p
            return (self.left if x[self.column] <= self.trashhold else self.right).predict(x, maxlvl - 1)
            
    def build_tree(self, df, levels, f):
        p = df['label'].sum() / len(df['label'].index)
        if levels == 0:
            return BinaryDecisionTree.Node(p)
        
        best_column = None
        best_trashhold = None
        best_ig = None
        
        for column in df.drop(columns=['label']).columns:
            for trashhold in df[column].unique():
                left_df = df[df[column] <= trashhold]
                right_df = df[df[column] > trashhold]
                
                left_df_sz = len(left_df.index)
                right_df_sz = len(right_df.index)
                
                if (left_df_sz == 0 or right_df_sz == 0):
                    continue
                
                cnt_left_1 = left_df['label'].sum()
                cnt_left_0 = left_df_sz - cnt_left_1
                cnt_right_1 = right_df['label'].sum()
                cnt_right_0 = right_df_sz - cnt_right_1
                
                ig = left_df_sz * f([cnt_left_0 / left_df_sz, cnt_left_1 / left_df_sz]) + \
                     right_df_sz * f([cnt_right_0 / right_df_sz, cnt_right_1 / right_df_sz])
                if best_ig is None or ig < best_ig:
                    best_ig = ig
                    best_column = column
                    best_trashhold = trashhold
        
        if best_ig is None:
            return BinaryDecisionTree.Node(p)
        
        assert best_column is not None
        return BinaryDecisionTree.Node(p,
                    left= self.build_tree(df[df[best_column] <= best_trashhold], levels - 1, f),
                    right=self.build_tree(df[df[best_column] > best_trashhold], levels - 1, f),
                    column=best_column, trashhold=best_trashhold)
        
    def __init__(self, df, levels, f):
        self.root = self.build_tree(df, levels, f)
        
    def predict(self, x, maxlvl):
        return self.root.predict(x, maxlvl)

In [ ]:
max_depth = 10

cancer_decision_tree = BinaryDecisionTree(cancer_train_set, max_depth, lambda p: p[0] * p[1])

In [66]:
cancer_best_depth = None
cancer_best_tree_auc = None

for depth in range(max_depth + 1):
    auc = sklearn.metrics.roc_auc_score(cancer_test_set['label'], [cancer_decision_tree.predict(cancer_test_set.iloc[i], maxlvl=depth) for i in range(len(cancer_test_set.index))])
    print("Cancer: AUC for depth {}: {}".format(depth, auc))
    if cancer_best_tree_auc is None or auc > cancer_best_tree_auc:
        cancer_best_tree_auc = auc
        cancer_best_depth = depth

Cancer: AUC for depth 0: 0.5
Cancer: AUC for depth 1: 0.8604729729729729
Cancer: AUC for depth 2: 0.9462837837837837
Cancer: AUC for depth 3: 0.9589527027027027
Cancer: AUC for depth 4: 0.9356418918918918
Cancer: AUC for depth 5: 0.8954391891891893
Cancer: AUC for depth 6: 0.9266891891891893
Cancer: AUC for depth 7: 0.9287162162162163
Cancer: AUC for depth 8: 0.9287162162162163
Cancer: AUC for depth 9: 0.9287162162162163
Cancer: AUC for depth 10: 0.9287162162162163


In [71]:
print("Best depth:", cancer_best_depth)

tree_cancer_tpr, tree_cancer_fpr, _ = sklearn.metrics.roc_curve(cancer_test_set['label'], [cancer_decision_tree.predict(cancer_test_set.iloc[i], maxlvl=cancer_best_depth) for i in range(len(cancer_test_set.index))])

Best depth: 3


In [72]:
draw_roc(list(zip(tree_cancer_fpr, tree_cancer_tpr)))

In [73]:
spam_decision_tree = BinaryDecisionTree(spam_train_set, max_depth, lambda p: p[0] * p[1])

In [74]:
spam_best_depth = None
spam_best_tree_auc = None

for depth in range(max_depth + 1):
    auc = sklearn.metrics.roc_auc_score(spam_test_set['label'], [spam_decision_tree.predict(spam_test_set.iloc[i], maxlvl=depth) for i in range(len(spam_test_set.index))])
    print("Spam: AUC for depth {}: {}".format(depth, auc))
    if spam_best_tree_auc is None or auc > spam_best_tree_auc:
        spam_best_tree_auc = auc
        spam_best_depth = depth

Spam: AUC for depth 0: 0.5
Spam: AUC for depth 1: 0.7362760668415963
Spam: AUC for depth 2: 0.8265112356225461
Spam: AUC for depth 3: 0.9044700810858799
Spam: AUC for depth 4: 0.9263765857123128
Spam: AUC for depth 5: 0.9263395939787322
Spam: AUC for depth 6: 0.9266207311539447
Spam: AUC for depth 7: 0.9350375836013178
Spam: AUC for depth 8: 0.9304678714463275
Spam: AUC for depth 9: 0.9217501528991656
Spam: AUC for depth 10: 0.9159819085761636


In [75]:
print("Best depth:", spam_best_depth)

tree_spam_tpr, tree_spam_fpr, _ = sklearn.metrics.roc_curve(spam_test_set['label'], [spam_decision_tree.predict(spam_test_set.iloc[i], maxlvl=spam_best_depth) for i in range(len(spam_test_set.index))])

Best depth: 7


In [76]:
draw_roc(list(zip(tree_spam_fpr, tree_spam_tpr)))